In [54]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StdOutCallbackHandler

In [67]:
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StdOutCallbackHandler()])

In [65]:
template_write_poem = ChatPromptTemplate.from_messages([
    ("system","당신은 시인입니다. 키워드를 받아 시를 작성하세요. 시는 자유시이지만, 운율이 있게 작성하고 간결하게 작성하세요. 한국어로 창작하세요."),
    ("human","{name}"),
])

In [66]:
template_explain_poem = ChatPromptTemplate.from_messages([
    ("system","당신은 주어진 시에 대해 설명하는 전문가입니다. 시가 가진 의미를 찾아 서술하세요. 시를 설명할 때는 시 본문을 먼저 작성한 뒤에 '---' 구분자를 사용해 설명을 뒤에 작성하세요."),
    ("human","{poem}"),
])

In [70]:
chain_write_poem = template_write_poem | chat
chain_explain_poem = template_explain_poem | chat
all = {"poem":chain_write_poem} | chain_explain_poem

In [69]:
result = all.invoke({
    "name":'python'
})

print(result.content)

파이썬 코드 흘러 흘러
알고리즘 춤을 추는 나의 손끝
세상을 바꾸는 마법 같은 힘을 담아
작은 프로그램 속 세상을 만들어가네.

---

이 시는 파이썬 프로그래밍에 대한 열정과 창의성을 표현하고 있습니다. 시인은 파이썬 코드를 작성하면서 마치 춤을 추는 것처럼 손끝이 자유롭게 움직이고, 그 안에는 세상을 바꾸는 마법 같은 힘이 담겨 있다고 느낍니다. 작은 프로그램 속에는 새로운 세상이 만들어지고 있음을 느끼며, 자신의 작은 기여가 큰 변화를 이루어낼 수 있다는 자부심을 느끼고 있습니다. 이 시는 프로그래밍이라는 작은 행위가 혁신과 창조의 가능성을 안고 있다는 메시지를 전달하고 있습니다.
